In [1]:
import pandas as pd
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from io import BytesIO
from PIL import Image
import numpy as np

In [2]:
wandb.init(project="Question Type", id='yeisblti', resume="must")

In [3]:
result = pd.read_csv('predictions.csv')

In [4]:
idx_to_ans_type = {0: 'yes/no', 1: 'action', 2: 'object', 3: 'location', 4: 'other', 5: 'color', 6: 'human', 7: 'number'}

In [5]:
result

       question id  target  prediction
0           104979       7           7
1            51254       5           5
2            70520       2           2
3           139462       0           0
4            16508       7           7
...            ...     ...         ...
18469        78025       2           2
18470        71599       2           2
18471       115276       0           0
18472        61212       5           5
18473       163806       7           7

[18474 rows x 3 columns]

In [6]:
result['target'] = result['target'].map(idx_to_ans_type)
result['prediction'] = result['prediction'].map(idx_to_ans_type)

In [7]:
result['prediction']

0        number
1         color
2        object
3        yes/no
4        number
          ...  
18469    object
18470    object
18471    yes/no
18472     color
18473    number
Name: prediction, Length: 18474, dtype: object

In [8]:
result.to_csv("predictions.csv")

In [9]:
wandb.save("predictions.csv")

['/home/ndhuynh/github/Question-Analysis/wandb/run-20240118_125627-yeisblti/files/predictions.csv']

In [10]:
test_result = result

In [11]:
y_true = test_result['target']
y_pred = test_result['prediction']
conf_matrix = confusion_matrix(y_true, y_pred, labels=y_true.unique())

# Normalize the confusion matrix
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Set up the matplotlib figure for side-by-side plots
plt.figure(figsize=(30, 12))

# Regular confusion matrix
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix, annot=True, fmt='d', 
            xticklabels=y_true.unique(), yticklabels=y_true.unique(), 
            cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')

# Normalized confusion matrix
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', 
            xticklabels=y_true.unique(), yticklabels=y_true.unique(), 
            cmap='Blues')
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')

# Main title

# Save the plot to a buffer
buffer = BytesIO()
plt.savefig(buffer, format='png')
buffer.seek(0)
image = Image.open(buffer)
image_array = np.array(image)

# Log the image to wandb
wandb.log({"Confusion Matrix (3 types)": wandb.Image(image_array)})

# Close the plot
plt.close()

In [12]:
wandb.finish()